In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Schizophrenia"
cohort = "GSE285666"

# Input paths
in_trait_dir = "../../input/GEO/Schizophrenia"
in_cohort_dir = "../../input/GEO/Schizophrenia/GSE285666"

# Output paths
out_data_file = "../../output/preprocess/Schizophrenia/GSE285666.csv"
out_gene_data_file = "../../output/preprocess/Schizophrenia/gene_data/GSE285666.csv"
out_clinical_data_file = "../../output/preprocess/Schizophrenia/clinical_data/GSE285666.csv"
json_path = "../../output/preprocess/Schizophrenia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Exon- and gene-Level transcriptional profiling in Lymphoblastoid Cell Lines (LCLs) from Williams syndrome patients and controls"
!Series_summary	"Williams syndrome (WS), characterized by positive sociality, provides a unique model for studying transcriptional networks underlying social dysfunction, relevant to disorders like autism spectrum disorder (ASD) and schizophrenia (SCHZ). In a cohort lymphoblastoid cell lines derived from 52 individuals (34 WS patients, 18 parental controls), genome-wide exon-level arrays identified a core set of differentially expressed genes (DEGs), with WS-deleted genes ranking among the top transcripts. Findings were validated by PCR, RNA-seq, and western blots."
!Series_summary	"Network analyses revealed perturbed actin cytoskeletal signaling in excitatory dendritic spines, alongside interactions in MAPK, IGF1-PI3K-AKT-mTOR/insulin, and synaptic actin pathways. These transcriptional networks show parallels to ASD and

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from Affymetrix Human Exon arrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at the sample characteristics dictionary
# The trait in this case is Schizophrenia, but the data appears to be about Williams syndrome vs controls
# This dataset is comparing Williams syndrome patients to controls, not specifically looking at Schizophrenia
trait_row = 0  # The disease state is in row 0, but it's for Williams syndrome, not Schizophrenia
age_row = None  # No age information provided
gender_row = None  # No gender information provided

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait information to binary (0 for control, 1 for case)"""
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() in ['unaffected parental control', 'control']:
        return 0
    elif 'williams syndrome patient' in value.lower():
        return 1
    else:
        return None

def convert_age(value):
    """Convert age information to continuous"""
    # Not used since age_row is None
    return None

def convert_gender(value):
    """Convert gender information to binary (0 for female, 1 for male)"""
    # Not used since gender_row is None
    return None

# 3. Save Metadata
# Trait data availability is determined by whether trait_row is not None
# However, in this case trait_row is for Williams syndrome, not Schizophrenia
# So we should set is_trait_available to False for our Schizophrenia study
is_trait_available = False  # The dataset doesn't contain Schizophrenia trait data

# Initial filtering on usability and save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since the trait data for Schizophrenia is not available, we skip this step


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Schizophrenia/GSE285666/GSE285666_series_matrix.txt.gz
Gene data shape: (22011, 52)
First 20 gene/probe identifiers:
Index(['2315554', '2315633', '2315674', '2315739', '2315894', '2315918',
       '2315951', '2316218', '2316245', '2316379', '2316558', '2316605',
       '2316746', '2316905', '2316953', '2317246', '2317317', '2317434',
       '2317472', '2317512'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be probe IDs/numeric identifiers rather than standard human gene symbols
# Human gene symbols typically follow patterns like BRCA1, TP53, etc.
# These numeric identifiers would need to be mapped to actual gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check if there are any columns that might contain gene information
sample_row = gene_annotation.iloc[0].to_dict()
print("\nFirst row as dictionary:")
for col, value in sample_row.items():
    print(f"{col}: {value}")

# Check if IDs in gene_data match IDs in annotation
print("\nComparing gene data IDs with annotation IDs:")
print("First 5 gene data IDs:", gene_data.index[:5].tolist())
print("First 5 annotation IDs:", gene_annotation['ID'].head().tolist())

# Properly check for exact ID matches between gene data and annotation
gene_data_ids = set(gene_data.index)
annotation_ids = set(gene_annotation['ID'].astype(str))
matching_ids = gene_data_ids.intersection(annotation_ids)
id_match_percentage = len(matching_ids) / len(gene_data_ids) * 100 if len(gene_data_ids) > 0 else 0

print(f"\nExact ID match between gene data and annotation:")
print(f"Matching IDs: {len(matching_ids)} out of {len(gene_data_ids)} ({id_match_percentage:.2f}%)")

# Check which columns might contain gene symbols for mapping
potential_gene_symbol_cols = [col for col in gene_annotation.columns 
                             if any(term in col.upper() for term in ['GENE', 'SYMBOL', 'NAME'])]
print(f"\nPotential columns for gene symbols: {potential_gene_symbol_cols}")

# Check if the identified columns contain non-null values
for col in potential_gene_symbol_cols:
    non_null_count = gene_annotation[col].notnull().sum()
    non_null_percent = non_null_count / len(gene_annotation) * 100
    print(f"Column '{col}': {non_null_count} non-null values ({non_null_percent:.2f}%)")



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_LIST', 'SPOT_ID', 'seqname', 'RANGE_GB', 'RANGE_STRAND', 'RANGE_START', 'RANGE_STOP', 'total_probes', 'gene_assignment', 'mrna_assignment', 'category']
{'ID': ['2315100', '2315106', '2315109', '2315111', '2315113'], 'GB_LIST': ['NR_024005,NR_034090,NR_024004,AK093685', 'DQ786314', nan, nan, 'DQ786265'], 'SPOT_ID': ['chr1:11884-14409', 'chr1:14760-15198', 'chr1:19408-19712', 'chr1:25142-25532', 'chr1:27563-27813'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['11884', '14760', '19408', '25142', '27563'], 'RANGE_STOP': ['14409', '15198', '19712', '25532', '27813'], 'total_probes': ['20', '8', '4', '4', '4'], 'gene_assignment': ['NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify columns for mapping
# ID column contains probe identifiers matching the gene expression data
# gene_assignment column contains gene symbols information
print("Extracting gene mapping from gene annotation data...")

# 2. Create gene mapping dataframe
# Using get_gene_mapping function from the library to extract the mapping columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("First few rows of gene mapping:")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# Use the apply_gene_mapping function from the library
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data after mapping, shape: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(gene_data.index[:10].tolist())

# Save the processed gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Extracting gene mapping from gene annotation data...


Gene mapping dataframe shape: (316481, 2)
First few rows of gene mapping:
        ID                                               Gene
0  2315100  NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-As...
1  2315106                                                ---
2  2315109                                                ---
3  2315111                                                ---
4  2315113                                                ---


Gene expression data after mapping, shape: (48895, 52)
First few gene symbols after mapping:
['A-', 'A-2', 'A-52', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0', 'A1']


Gene expression data saved to ../../output/preprocess/Schizophrenia/gene_data/GSE285666.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
# Using the selected_clinical_df directly for proper trait information
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print(f"Selected clinical data shape: {selected_clinical_df.shape}")
print("Clinical data preview:")
print(selected_clinical_df)

# Link clinical and genetic data directly using the selected clinical dataframe
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5] if not linked_data.empty else "Linked data is empty")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in features
try:
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
except Exception as e:
    print(f"Error checking for bias: {e}")
    is_biased = True  # Assume biased if there's an error

# 5. Validate and save cohort information - setting is_trait_available to False as this dataset 
# contains Williams syndrome data, not Schizophrenia data
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=False,  # Changed to False as dataset doesn't contain Schizophrenia
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains Williams syndrome patients vs controls, not Schizophrenia data."
)

# 6. Save the linked data if usable
if is_usable and not linked_data.empty:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")

Gene data shape after normalization: (18418, 52)


Normalized gene expression data saved to ../../output/preprocess/Schizophrenia/gene_data/GSE285666.csv
Selected clinical data shape: (1, 52)
Clinical data preview:
               GSM8706502  GSM8706503  GSM8706504  GSM8706505  GSM8706506  \
Schizophrenia         0.0         0.0         0.0         0.0         0.0   

               GSM8706507  GSM8706508  GSM8706509  GSM8706510  GSM8706511  \
Schizophrenia         0.0         0.0         0.0         0.0         0.0   

               ...  GSM8706544  GSM8706545  GSM8706546  GSM8706547  \
Schizophrenia  ...         1.0         1.0         1.0         1.0   

               GSM8706548  GSM8706549  GSM8706550  GSM8706551  GSM8706552  \
Schizophrenia         1.0         1.0         1.0         1.0         1.0   

               GSM8706553  
Schizophrenia         1.0  

[1 rows x 52 columns]
Linked data shape: (52, 18419)
Linked data preview (first 5 rows, 5 columns):
            Schizophrenia       A1BG   A1BG-AS1       A1CF         A2M
GS

Data shape after handling missing values: (52, 18419)
For the feature 'Schizophrenia', the least common label is '0.0' with 18 occurrences. This represents 34.62% of the dataset.
The distribution of the feature 'Schizophrenia' in this dataset is fine.

Dataset is not usable for analysis. No linked data file saved.
